In [1]:
import pandas as pd


# Object
`이용자의 구독 기간, 로그인 활동, 학습 세션 참여도`와 같은 데이터 분석을 통해 학습 플랫폼 이용자의 구독 갱신 여부를 예측하는 AI 모델을 개발
***

## 자료 분석 및 전처리
| column | Description |
|------|---------------|
| user_id | 사용자의 고유 식별자 |
| subscription_duration | 사용자가 서비스에 가입한 기간 (월) |
| recent_login_time | 사용자가 마지막으로 로그인한 시간 (일) |
| average_login_time | 사용자의 일반적인 로그인 시간 |
| average_time_per_learning_session | 각 학습 세션에 소요된 평균 시간 (분) |
| monthly_active_learning_days | 월간 활동적인 학습 일수 |
| total_completed_courses | 완료한 총 코스 수 |
| recent_learning_achievement | 최근 학습 성취도 |
| abandoned_learning_sessions | 중단된 학습 세션 수 |
| community_engagement_level | 커뮤니티 참여도
| preferred_difficulty_level | ~~선호하는 난이도~~  `Low : 1, Medium : 2, High : 3`|
| subscription_type | ~~구독 유형~~  `Basic : 1, Premium : 2` |
| customer_inquiry_history | 고객 문의 이력 |
| payment_pattern | 사용자의 지난 3개월 간의 결제 패턴을 10진수로 표현한 값.<br> - 7: 3개월 모두 결제함<br> - 6: 첫 2개월은 결제했으나 마지막 달에는 결제하지 않음<br>- 5: 첫 달과 마지막 달에 결제함<br>- 4: 첫 달에만 결제함<br>- 3: 마지막 2개월에 결제함<br>- 2: 가운데 달에만 결제함<br> - 1: 마지막 달에만 결제함<br>- 0: 3개월 동안 결제하지 않음 |
| target | 사용자가 다음 달에도 구독을 계속할지 (1) 또는 취소할지 (0)를 나타냅니다.|

<br>
<code>
<접근 방향><br>
1. 타겟 1과 0을 일단 나눠서 각 특성들과 상관 분석해보기<br>
1-1. 주어진 기본 특성<br>
1-2. 파생 특성<br>
2. 지도 학습 / 이진 분류 / 다중 분류
</code>

In [2]:
train_df = pd.read_csv('./raw data/train.csv')
train_df.head()

,user_id,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern,target
0,b919c29d,13,14,14.946163,8.427187,18,16,68.360455,3,4,Low,Basic,4,5,0
1,a0a60abb,16,18,18.453224,72.646087,16,13,97.567322,2,3,Medium,Basic,1,6,1
2,b9f171ae,22,1,16.195228,21.774492,13,14,94.358763,3,4,Medium,Premium,0,7,1
3,5dc0ba8b,1,19,17.628656,42.659066,19,18,70.153228,0,3,Low,Basic,1,0,1
4,65c83654,4,5,21.390656,30.744287,19,10,81.917908,2,4,Medium,Basic,3,0,1


In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   user_id                            10000 non-null  object 
 1   subscription_duration              10000 non-null  int64  
 2   recent_login_time                  10000 non-null  int64  
 3   average_login_time                 10000 non-null  float64
 4   average_time_per_learning_session  10000 non-null  float64
 5   monthly_active_learning_days       10000 non-null  int64  
 6   total_completed_courses            10000 non-null  int64  
 7   recent_learning_achievement        10000 non-null  float64
 8   abandoned_learning_sessions        10000 non-null  int64  
 9   community_engagement_level         10000 non-null  int64  
 10  preferred_difficulty_level         10000 non-null  object 
 11  subscription_type                  10000 non-null  obje

target data : 10,000개 데이터, 결측치 X

In [10]:
# label encoder할 object 종류 확인
print(train_df['preferred_difficulty_level'].unique())
print(train_df['subscription_type'].unique())

['Low' 'Medium' 'High']
['Basic' 'Premium']


In [ ]:
레이블 인코더링 하기

## 2. 기본 특성들로 예측하기